In [1]:
import cv2, numpy as np, os

In [2]:
working_dir = os.getcwd()

In [3]:
path = os.path.join(working_dir,'../images/shapes')
os.chdir(path)
# print(os.getcwd())
img_size = 60 

In [4]:
folders = os.listdir(os.getcwd())
labels, images = [], []
print(folders)

['circle', 'square', 'triangle']


In [5]:
for folder in folders:
#     if (folder == 'star'):
#         continue
    print (folder)
    for path in os.listdir(os.getcwd()+'/'+folder):
        img = cv2.imread(folder+'/'+path,0)
        #cv2.imshow('img', img)
        #cv2.waitKey(1)
        images.append(cv2.resize(img, (img_size, img_size)))
        labels.append(folders.index(folder))

circle
square
triangle


In [6]:
# to_train= 0
# train_images, test_images, train_labels, test_labels = [],[],[],[]
# for image, label in zip(images, labels):
#     if to_train<5:
#         train_images.append(image)
#         train_labels.append(label)
#         to_train+=1
#     else:
#         test_images.append(image)
#         test_labels.append(label)
#         to_train = 0

from sklearn.model_selection import train_test_split

images = np.array(images)
labels = np.array(labels)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

print(len(train_images), len(test_images), len(train_labels), len(test_labels))

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

8964 2241 8964 2241
(8964, 60, 60) (2241, 60, 60) (8964,) (2241,)


In [7]:
print(len(images))
# print(images[11000])

11205


In [8]:
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(len(images), dimData)
    images = images.astype('float32')
    images /=255
    return images

In [9]:
#flatten data
dataDim = np.prod(images[0].shape)

train_data = flatten(dataDim, train_images)
test_data  = flatten(dataDim, test_images)

In [10]:
from keras.utils import to_categorical

train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

In [11]:
print(train_labels_one_hot[2604])

[1. 0. 0.]


In [12]:
classes = np.unique(train_labels)
nClasses  = len(classes)
print(classes, nClasses)

[0 1 2] 3


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [14]:
model = Sequential()
model.add(Dense(256, activation = 'tanh', input_shape = (dataDim,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_one_hot, batch_size = 256, epochs=50, verbose=1,
                    validation_data=(test_data, test_labels_one_hot))

Epoch 1/50
36/36 [==============================] - 2s 49ms/step - loss: 1.2560 - accuracy: 0.3421 - val_loss: 1.4128 - val_accuracy: 0.3253
Epoch 2/50
36/36 [==============================] - 1s 36ms/step - loss: 1.0306 - accuracy: 0.4615 - val_loss: 0.6811 - val_accuracy: 0.6600
Epoch 3/50
36/36 [==============================] - 1s 36ms/step - loss: 0.7201 - accuracy: 0.6099 - val_loss: 0.4928 - val_accuracy: 0.6600
Epoch 4/50
36/36 [==============================] - 1s 37ms/step - loss: 0.7386 - accuracy: 0.6487 - val_loss: 0.5374 - val_accuracy: 0.6600
Epoch 5/50
36/36 [==============================] - 1s 37ms/step - loss: 0.4472 - accuracy: 0.7829 - val_loss: 0.4202 - val_accuracy: 0.7340
Epoch 6/50
36/36 [==============================] - 1s 38ms/step - loss: 0.4759 - accuracy: 0.8099 - val_loss: 0.3559 - val_accuracy: 0.8581
Epoch 7/50
36/36 [==============================] - 1s 37ms/step - loss: 0.3207 - accuracy: 0.8670 - val_loss: 1.0180 - val_accuracy: 0.6600
Epoch 8/50
36

In [16]:
[test_loss, test_acc] = model.evaluate(test_data, test_labels_one_hot)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

71/71 [==============================] - 0s 4ms/step - loss: 0.0126 - accuracy: 0.9987
Evaluation result on Test Data : Loss = 0.012564926408231258, accuracy = 0.9986613392829895


In [27]:
model_path = os.path.join(os.getcwd(),'../../model')
os.chdir(model_path)
# print(os.getcwd())

model.save('shapesmodel.h5')